In [2]:
!pip install pandas numpy scikit-learn matplotlib seaborn jupyter pyarrow



Defaulting to user installation because normal site-packages is not writeable
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached h11-0.16.0-py3-none-any.whl (37 kB)

  Attempting uninstall: h11

    Found existing installation: h11 0.9.0

    Uninstalling h11-0.9.0:

      Successfully uninstalled h11-0.9.0

   ---------------------------------------- 0/3 [h11]
   ---------------------------------------- 0/3 [h11]
   ---------------------------------------- 0/3 [h11]
   ---------------------------------------- 0/3 [h11]
  Attempting uninstall: httpcore
   ---------------------------------------- 0/3 [h11]
    Found existing installation: httpcore 0.9.1
   ---------------------------------------- 0/3 [h11]
   ------------- --------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.28.1 which is incompatible.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install sqlalchemy pymysql

Defaulting to user installation because normal site-packages is not writeable
  Using cached pymysql-1.1.2-py3-none-any.whl.metadata (4.3 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------  2.1/2.1 MB 18.6 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 13.8 MB/s  0:00:00
Using cached pymysql-1.1.2-py3-none-any.whl (45 kB)

   ---------------------------------------- 0/3 [pymysql]
   ---------------------------------------- 0/3 [pymysql]
   ---------------------------------------- 0/3 [pymysql]
   ---------------------------------------- 0/3 [pymysql]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# 3.1 Imports & data read

import pandas as pd
import numpy as np
from pathlib import Path

RAW = Path("../capstone")

cust = pd.read_csv(RAW/"customers.csv")
usage = pd.read_csv(RAW/"usage_data.csv")
comp = pd.read_csv(RAW/"complaints.csv", parse_dates=["created_at"])
bill = pd.read_csv(RAW/"billing.csv")

print(cust.shape, usage.shape, comp.shape, bill.shape)
cust.head()



(10, 4) (10, 4) (10, 4) (10, 5)


,customer_id,name,plan_type,region
0,1001,Asha Mehta,Prepaid,Delhi
1,1002,Ravi Kumar,Postpaid,Mumbai
2,1003,Sneha Rao,Prepaid,Chennai
3,1004,Manoj Singh,Postpaid,Delhi
4,1005,Divya Jain,Prepaid,Kolkata


In [7]:
# 3.2 Basic validation

for df,name in [(cust,"customers"),(usage,"usage"),(comp,"complaints"),(bill,"billing")]:
    print(name, "nulls:\n", df.isna().sum(), "\n")


customers nulls:
 customer_id    0
name           0
plan_type      0
region         0
dtype: int64 

usage nulls:
 customer_id     0
data_used_gb    0
calls_made      0
revenue_inr     0
dtype: int64 

complaints nulls:
 customer_id    0
category       0
created_at     0
status         0
dtype: int64 

billing nulls:
 customer_id        0
tenure             0
contract_type      0
monthly_charges    0
churn              0
dtype: int64 



In [8]:
# 3.3 Light cleaning (fixable—not dropping)

# Trim/case normalisation
cust["region"]    = cust["region"].str.title().str.strip()
cust["plan_type"] = cust["plan_type"].str.title().str.strip()

# Fill numeric gaps sensibly
for col in ["data_used_gb","calls_made","revenue_inr"]:
    usage[col] = usage[col].fillna(usage[col].median())

bill["monthly_charges"] = bill["monthly_charges"].fillna(bill["monthly_charges"].median())
bill["tenure"] = bill["tenure"].fillna(bill["tenure"].median())

# Fill complaints fields
comp["status"]   = comp["status"].fillna("Resolved")
comp["category"] = comp["category"].fillna("General")

# Convert churn to 0/1 flag
bill["churn_flag"] = (bill["churn"].str.lower() == "yes").astype(int)


In [9]:
# 3.4 Aggregate complaints to customer level

complaint_agg = (comp
   .groupby("customer_id")
   .agg(
       complaints_total = ("category","count"),
       complaints_open  = ("status", lambda s: (s.str.lower()=="open").sum())
   )
   .reset_index()
)
complaint_agg.head()


,customer_id,complaints_total,complaints_open
0,1002,2,2
1,1003,1,1
2,1004,1,1
3,1005,1,0
4,1006,1,0


In [10]:
# 3.5 Join all tables

df = (cust
      .merge(usage, on="customer_id", how="left")
      .merge(complaint_agg, on="customer_id", how="left")
      .merge(bill, on="customer_id", how="left"))

# Fill new NaNs created by left joins
df[["complaints_total","complaints_open"]] = df[["complaints_total","complaints_open"]].fillna(0).astype(int)

# Final sanity checks
print(df.isna().sum())
df.head()


customer_id         0
name                0
plan_type           0
region              0
data_used_gb        0
calls_made          0
revenue_inr         0
complaints_total    0
complaints_open     0
tenure              0
contract_type       0
monthly_charges     0
churn               0
churn_flag          0
dtype: int64


,customer_id,name,plan_type,region,data_used_gb,calls_made,revenue_inr,complaints_total,complaints_open,tenure,contract_type,monthly_charges,churn,churn_flag
0,1001,Asha Mehta,Prepaid,Delhi,5.2,25,180,0,0,12,Month-to-Month,180,Yes,1
1,1002,Ravi Kumar,Postpaid,Mumbai,12.5,40,280,2,2,24,One Year,280,No,0
2,1003,Sneha Rao,Prepaid,Chennai,7.8,32,210,1,1,8,Month-to-Month,210,Yes,1
3,1004,Manoj Singh,Postpaid,Delhi,15.6,55,320,1,1,30,One Year,320,No,0
4,1005,Divya Jain,Prepaid,Kolkata,3.4,18,120,1,0,6,Month-to-Month,120,Yes,1


In [11]:
# 3.6 Save master dataset

OUT = Path("../outputs")
OUT.mkdir(exist_ok=True)
df.to_csv(OUT/"telecom_master.csv", index=False)
print("Saved:", OUT/"telecom_master.csv")



Saved: ..\outputs\telecom_master.csv


In [ ]:
# 4) Create Churn Model

import pandas as pd, seaborn as sns, matplotlib.pyplot as plt
df = pd.read_csv("../outputs/telecom_master.csv")

# Churn rate
print(df["churn_flag"].mean().round(3))   # e.g., 0.23 -> 23%

# Churn by plan/region
sns.barplot(x="plan_type", y="churn_flag", data=df); plt.title("Churn by Plan"); plt.show()
sns.barplot(x="region", y="churn_flag", data=df);    plt.title("Churn by Region"); plt.xticks(rotation=30); plt.show()

# Revenue vs churn
sns.boxplot(x="churn_flag", y="monthly_charges", data=df); plt.title("Charges vs Churn"); plt.show()

# Correlations (numeric only)
num = df.select_dtypes("number")
sns.heatmap(num.corr(), cmap="vlag", center=0); plt.title("Numeric Corr"); plt.show()



In [ ]:
# 5.1 Setup features

import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
import joblib, json, matplotlib.pyplot as plt, seaborn as sns

df = pd.read_csv("../outputs/telecom_master.csv")

# Choose simple, strong baseline features
features = [
    "tenure","monthly_charges","data_used_gb","calls_made","revenue_inr",
    "complaints_total","complaints_open",
    "contract_type","plan_type","region"
]
target = "churn_flag"

X = df[features].copy()
y = df[target].copy()

cat_cols = ["contract_type","plan_type","region"]
num_cols = [c for c in X.columns if c not in cat_cols]


In [ ]:
# 5.2 Preprocessor + models

pre = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

logit = Pipeline([
  ("prep", pre),
  ("clf", LogisticRegression(max_iter=1000, class_weight="balanced"))
])

tree = Pipeline([
  ("prep", pre),
  ("clf", DecisionTreeClassifier(max_depth=6, class_weight="balanced", random_state=42))
])


In [ ]:
# 5.3 Train/test split & training

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

logit.fit(X_train, y_train)
tree.fit(X_train, y_train)


In [ ]:
5.4 Evaluate
# 
def evaluate(name, model):
    pred = model.predict(X_test)
    proba = getattr(model, "predict_proba", lambda X: np.c_[1-pred, pred])(X_test)[:,1]
    acc = accuracy_score(y_test, pred)
    p, r, f1, _ = precision_recall_fscore_support(y_test, pred, average="binary")
    print(f"{name} -> Acc:{acc:.3f}  Prec:{p:.3f}  Rec:{r:.3f}  F1:{f1:.3f}")

    cm = confusion_matrix(y_test, pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"{name} Confusion Matrix"); plt.xlabel("Pred"); plt.ylabel("True"); plt.show()
    return acc, p, r, f1

metrics = {}
for nm, mdl in [("Logistic", logit), ("DecisionTree", tree)]:
    acc,p,r,f1 = evaluate(nm, mdl)
    metrics[nm] = dict(accuracy=acc, precision=p, recall=r, f1=f1)

with open("../outputs/model_metrics.json","w") as f:
    json.dump(metrics, f, indent=2)
metrics

Goal: Hit a reasonable baseline (e.g., accuracy 0.70–0.85, recall decent for catching churn).
If recall is low, try deeper tree (max_depth=8), or add/engineer features (e.g., ARPU = revenue_inr / max(1, tenure)).


In [ ]:
# 5.5 Pick the winner & persist

best = logit if metrics["Logistic"]["f1"] >= metrics["DecisionTree"]["f1"] else tree
joblib.dump(best, "../outputs/model.pkl")
print("Saved best model.")


In [ ]:
# 5.6 Export predictions for BI

best = joblib.load("../outputs/model.pkl")
df_pred = df.copy()
df_pred["churn_prob"] = best.predict_proba(X)[:,1]
df_pred["churn_pred"] = (df_pred["churn_prob"] >= 0.5).astype(int)
df_pred.to_csv("../outputs/predictions.csv", index=False)
print("Saved predictions.csv")
